# CONFIGURAÇÕES

In [1]:
import os
import sys
import importlib
from pathlib import Path

import pandas as pd
import geopandas as gpd

import numpy as np



import matplotlib.pyplot as plt 
import contextily as cx

In [2]:


gpd.options.io_engine = "pyogrio"
os.environ["PYOGRIO_USE_ARROW"] = "1"

from pyogrio import list_layers

## Definição da diretoria de trabalho na directoria raiz do projeto

In [3]:
from pathlib import Path

# Find the project root (assuming marker-based or script-relative path)
def find_project_root(marker="README.md"):
    current_dir = Path.cwd()
    while current_dir != current_dir.parent:  # Traverse up until root
        if (current_dir / marker).exists():
            return current_dir
        current_dir = current_dir.parent
    raise FileNotFoundError(f"Marker '{marker}' not found in any parent directory.")

project_root = find_project_root()
sys.path.append(str(project_root)) 

# Or use a relative path: project_root = Path(__file__).resolve().parent.parent
os.chdir(project_root)
print(f"Working directory set to: {project_root}")

Working directory set to: c:\Users\paulo\OneDrive\ONEDRIVE_CLOUD_DISK\TRABALHO_AULAS\AL20242025\2SEM\ETE_2425\TP\projETE2425


## ADD our own library to the libraries paths

In [4]:
module_path = os.path.abspath(os.path.join(r'.\src'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [6]:
module_path

'c:\\Users\\paulo\\OneDrive\\ONEDRIVE_CLOUD_DISK\\TRABALHO_AULAS\\AL20242025\\2SEM\\ETE_2425\\TP\\projETE2425\\src'

In [7]:
import utils.utils_ete2425

## Diretorias de interesse

# Ler dados de base

In [7]:
print(list_layers(r'.\data\input\geopackage_ETE2425.gpkg'))

[['BGRI2021_1312' 'MultiPolygon']
 ['AE_AVRILH_MACRO_12UTs' 'MultiPolygon']
 ['AE_AVRILH_CAOP2018_FREG' 'MultiPolygon']
 ['WORLD_COUNTRIES' 'MultiPolygon']
 ['AE_AVRILH_MICRO_50UTs' 'MultiPolygon']
 ['AE_AVRILH_MESO_4UTs_M2' 'MultiPolygon']
 ['AE_AVRILH_CAOP2018_MUNI' 'MultiPolygon']
 ['BGRI2021_0105e0110' 'MultiPolygon']
 ['BGRI2011_0105e0110' 'MultiPolygon']
 ['gdf_BGRI2021_1312' 'MultiPolygon']
 ['gdf_CircEUlar_selectF' 'MultiPolygon']]


In [ ]:
# Leitura da base de dados inicial

# -------------------------------
# NOTA SOBRE O CONJUNTO DE DADOS
# -------------------------------
# Dados para a construção de um modelo de "Circularidade" do ambiente urbano do municipio do Porto
# Circularidade: consumo de energia e emissões de dioxido de carbono associadas 
# a i) consumo de energia "operacional" do parque habitacional, ii) iii) e iv) necessidades de energia nominal 
# para aquecimento, arrefecimento e produção de águas quentes sanitárias no parque habitacional, v) vi) consumo de energia 
# para satisfação das necessidades de mobilidade, em transporte individual e em transporte públicos e vii) viii) energia embutida
# em materiais de construção da habitação e das infraestruturas urbanas.
# Para explicar os padrões espaciais das variáveis de "circularidade" referidas anteriormente 
# considera-se dimensões associadas às características do parque habitacional, 
# da mobilidade, da morfologia urbana e das caracteristicas socioeconómicas da população residente.
# -------------------------------

circEUlar_gdb = gpd.read_file(r'.\data\input\geopackage_ETE2425.gpkg', 
                             layer='gdf_CircEUlar_selectF')


In [ ]:
circEUlar_gdb.columns

## Dados administrativos nacionais 

In [ ]:
# A "Carta Administrativa Oficial de Portugal" pode ser obtida em: https://www.dgterritorio.gov.pt/cartografia/cartografia-tematica/caop?language=en

print(list_layers(r'.\data\input' + r'\CAOP2023\Continente_CAOP2023.gpkg'))

In [ ]:
shape_CAOP_MUNICIPIOS = gpd.read_file(r'.\data\input' + r'\CAOP2023\Continente_CAOP2023.gpkg', layer = 'Cont_Mun_CAOP2023' )

In [ ]:
shape_CAOP_MUNICIPIOS.plot()

In [ ]:
shape_CAOP_MUNICIPIOS.head()

## Dados modelo CircEUlar

In [ ]:
circEUlar_gdb.head(3)

In [ ]:
# Leitura do ficheiro geoespacial referente à informação geométrica das subsecções estatísticas para o município do Porto
shape_BGRI21_Porto = gpd.read_file(r'.\data\input\geopackage_ETE2425.gpkg', 
                             layer='BGRI2021_1312')

In [ ]:
shape_BGRI21_Porto.head(3)

In [ ]:
shape_BGRI21_Porto.head(3)

In [ ]:
shape_BGRI21_Porto.geometry[1]

In [ ]:
shape_BGRI21_Porto.plot()

In [ ]:
shape_BGRI21_Porto.crs

In [ ]:
shape_BGRI21_Porto.shape

In [ ]:
shape_BGRI21_Porto.columns

In [ ]:
# Criar uma representação geométrica do municipio (polígono único)
# Permite-nos ter uma representação dos limites da área de estudo
# Atenção que mantém o mesmo referencial geográfico dos dados originais

shape_MUNICIPIO_Porto = shape_BGRI21_Porto.dissolve('DTMN21')
shape_MUNICIPIO_Porto.plot()

In [ ]:
# Cálculo da área de cada subsecção estatística

shape_BGRI21_Porto['AREA_SUBSECSAO'] = shape_BGRI21_Porto['geometry'].area

In [ ]:
shape_BGRI21_Porto[['BGRI2021', 'AREA_SUBSECSAO', 'SHAPE_Area' ]].head(3)

In [ ]:
# Projeção geográfica do sistema de referenciação fornecido na BGRI2021 (EPSG:3763) para o sistema de referenciação
# utilizado na biblioteca "contextily" (EPSG:3857)
# A biblioteca "contextily" permite a sobreposição de mapas base (imagens de satélite, mapas de ruas, etc) sobre
# os nossos mapas / representações geométricas das nossas unidades de análise

shape_BGRI21_Porto_reprojetct = shape_BGRI21_Porto.to_crs(epsg=3857)

In [ ]:
# Adição de um mapa de fundo ao nosso mapa de subsecções estatísticas do Porto
# Neste caso, associamos um mapa obtido no serviço OpenStreetMaps https://www.openstreetmap.org/

ax = shape_BGRI21_Porto_reprojetct.plot(figsize=(10, 10), alpha=0.2, edgecolor="k")
cx.add_basemap(ax)

## Combinar os dados da tabela CircEUlar com os dados geométricos

In [ ]:
shape_BGRI21_Porto.dtypes

In [ ]:
circEUlar_gdb.dtypes

In [ ]:
# Sendo a nossa unidade de análise a subsecção estatística
# Iremos utilizar o identificador geográfico da unidade geográfica "subsecção estatística"
# fornecido pela variável "BGRI2021" (combinação hierárquica de código de municipio, freguesia, secção e subsecção estatística) 
# para juntar a informação geoespacial (representação geométrica) com a informação alfanumérica da base de dados "circEUlar_db"
# Esta operação é baseada na combinação de tabelas usando a "chave" única definida pelo identificador geográfico "BGRI2021"
# Como o identificador geográfico é um código alfanumético, deve considerar-se definir este código
# como uma variável do tipo "String" em ambas as tabelas, por forma a melhor garantir a correta junção das tabelas


circEUlar_gdb['BGRI2021'] = circEUlar_gdb['BGRI2021'].astype(str)

In [ ]:
circEUlar_gdb.dtypes

In [ ]:
shape_BGRI21_Porto_CircEUlar = circEUlar_gdb.merge(shape_BGRI21_Porto.drop(columns=['geometry']), left_on = 'BGRI2021', right_on = 'BGRI2021', how = 'left')



In [ ]:
shape_BGRI21_Porto_CircEUlar.shape

In [ ]:
shape_BGRI21_Porto_CircEUlar.columns

In [ ]:
shape_BGRI21_Porto_CircEUlar.head(3)



# Análise exploratória (visualização de dados geográficos - POLIGONOS)

Os cartogramas constituem um instrumento fundamental em ciência de dados com dados espaciais, ao permiti explorar poteniciais padrões de distribuição geográfica das variáveis de interesse. 
Nos cartogramas podemos representar qualquer um dos objetos de análise e suas representações (ponto, linhas, polígonos) bem como podemos sobrepor vários tipos de objectos diferentes.
Uma das formas básicas de expandirmos os dados de base passa pela integração nos cartogramas de dados espaciais comuns, como é o caso das unidades político administrativas. 
A integração destes dados adicionais permite-nos observar padrões geográficos de duas formas:
- sobreposição de camadas para visualização, combinando uma camada com os dados iniciais e sua representação geométrica e outras camadas com as representações geométricas dos dados espaciais adicionais (por exemplo, adicionando limites administrativis).
- combinação de dados na tabela e informação (estatística) sumária condicional a uma dada divisão territorial (por exemplo, agregar e representar dados de uma variável como o consumo de energia para uma unidade geográfica distinta da original) 

A visualização de dados recorrendo a cartogramas beneficia da aplicação de técnicas simples de classificação, permitindo reduzir a carga cognitiva envolvida na analise de informação e fornecendo uma forma expedita de explorar os dados, incluindo a sua complexidade espacial. No entanto, a eficácia de um cartograma é condicional à escolha do esquema de classificação adotado e do conjunto de opções de desenho consideradas (por exemplo a cor ou estratégia de simbolização adotada). 

Nesta secção procuraremos explorar alguns destes detalhes, as suas implicações e o papel destas técnicas e ferramentas na estatística espacial. 

In [ ]:
figura, ax = plt.subplots(1, figsize=(9, 9))
shape_BGRI21_Porto_CircEUlar.plot(ax=ax, column='HousingPrice', 
                               legend=True) 
ax.set_axis_off()
ax.set_title('Preço €/m2 de transação de habitações (2023)')
plt.axis('equal')
plt.show()

## Notas sobre classificação dos dados para visualização

A classificação de dados para visualização constitui um problema de particionar os valores dos atributos em classes mutuamente exclusivas e exaustivas. A estratégia de partição é condicional à escala de medição do atributo em questão. Para atributos quantitativos (escalas ordinais, de intervalo, de razão) as classes terão uma ordenação explícita.
Mais formalmente, o problema de classificação é definir os limites de classe tal que:
$$
c_j < y_i \le  c_{j+1} \ \forall y_i \in C_{j}
$$

- onde $y_i$ é o valor do atributo, com localização espacial  $i$ (no caso de cartogramas com limites admistrativos, este índice corresponde à unidade administrativa; no caso de cartogramas de objetos de análise, este índice corresponde especificamente à localização desse mesmo objeto; note-se que os limites administrativos podem também ser considerados os nossos objetos de análise - tudo depende da formulação do problema.
- $j$ é o índice da classe 
- e $c_j$ representa o limite inferior do intervalo $j$.

Para operacionalizar este objetivo o pacote Paysal disponibiliza a livraria Mapclassify, devendo ser acedida a [documentação oficial](https://pysal.org/mapclassify/api.html)

In [ ]:
# https://pysal.org/mapclassify/api.html
# pysal.viz »» mapclassify 
import mapclassify as mc

import seaborn as sns

In [ ]:
shape_BGRI21_Porto_CircEUlar['HousingPrice'].describe().round(1)


A visualização dos padrões geográficos das variáveis constitui um elemento chave na análise exploratória dos dados espacias.
No entanto, a visualização requer a configuração de paletes de cor para descrever as características marcantes dos dados. Importa assim perceber a distribuição dos dados e fornecer uma palete de cores que nos descreva as propriedades mais marcantes dessa distribuição.

Importa assim conjugar a exploração gráfica da distribuição dos dados - para a qual recorremos à livraria mapclassify e seaborn - bem como a sua visualização geográfica.

In [ ]:
# Intervalos iguais

classi = mc.EqualInterval(shape_BGRI21_Porto_CircEUlar['HousingPrice'], k=7)
classi

In [ ]:
# Quantiles

classi_quant = mc.Quantiles(shape_BGRI21_Porto_CircEUlar['HousingPrice'], k=5)
classi_quant

In [ ]:
classi_quant.bins

In [ ]:
# Código padrão - usa a livraria seaborn (extende a matplotlib )
# https://seaborn.pydata.org/tutorial.html
# Set up the figure
f, ax = plt.subplots(1)
# Plot the kernel density estimation (KDE)
sns.kdeplot(shape_BGRI21_Porto_CircEUlar['HousingPrice'], fill=True)
# Add a blue tick for every value at the bottom of the plot (rugs)
sns.rugplot(shape_BGRI21_Porto_CircEUlar['HousingPrice'], alpha=0.5)
# Loop over each break point and plot a vertical red line
for cut in classi_quant.bins:
    plt.axvline(cut, color='red', linewidth=0.75)

In [ ]:
#NOTAs: 
# note-se que agora temos de definir os argumentos column, scheme e cmap
# o argumento «cmap» pode ser configurado seguindo esquemas predefinidos:
# https://matplotlib.org/stable/gallery/color/colormap_reference.html


#Usar a matplotlib para visualizar vários dados
figura, ax = plt.subplots(figsize = (10,10) )
ax.set_aspect('equal')
shape_MUNICIPIO_Porto.plot(ax=ax, zorder=2, 
                             color='grey', 
                             edgecolor='black', linewidth=2,
                            alpha = 0.3)

shape_BGRI21_Porto_CircEUlar.plot(ax=ax,zorder=1,
                      # marker="s",
                      column='HousingPrice', 
                      scheme='Quantiles', 
                      # k=7, 
                      cmap=plt.cm.Reds, 
                      linewidth=0 )

In [ ]:
# Desvio padrão médio

classi = mc.StdMean(shape_BGRI21_Porto_CircEUlar['HousingPrice'])
classi

In [ ]:
# Baseado na caixa de bigodes

classi = mc.BoxPlot(shape_BGRI21_Porto_CircEUlar['HousingPrice'])
classi

In [ ]:
# Fisher Jenks

classi = mc.FisherJenks(shape_BGRI21_Porto_CircEUlar['HousingPrice'])
classi

# Matriz de pesos e vizinhança W

As matrizes de pesos espaciais e vizinhança são um instrumento analítico básico em dados espaciais, permitindo descrever as relações geográficas entre as unidades observacionais referenciadas espacialmente. 
Ao expressar a noções de proximidade, vizinhança, conexão espacial ou, especificamente, conexões geográficas, a matriz de pesos espaciais constituem um dispositivo essencial em qualquer etapa de análise de dados espaciais, sendo inclusive integrados na fase de modelação, sendo parte de uma variada família de modelos espaciais - como veremos mais à frente. 

Numa perspetiva exploratória, as relações espaciais entre um objeto de análise específico e aqueles, objetos do mesmo tipo, que se encontram (“na sua vizinhança”, “na sua proximidade”) ou outros objectos relevantes, também eles circundantes (“mercearias”, "escola", "habitação dos avós"). 
Para uma abordagem estatística a estas relações espaicias importa estabelecer mecanismos formais e estruturados que permitem determinar as relações entre todos os pares de observações e objetos considerados. Isto significa que é necessário construir uma topologia - uma estrutura matemática que expressa as relações geométricas e espaciais entre observações. Este elemento é essencial pois permite fornecer uma topologia única, aplicável a todas as observações , tornando possível uma análise integrada, no espaço, de todos os elementos e não só especificamente de um objeto espacial.

Como facilmente se compreenderá, a construção e específicação deste dispositivo implica que toda a análise espacial subsequente está condicionada às especificações teóricas (concepções de "espaço") e matemáticas (formas de representar esse espaço - geralmente, implica questões de dimensionalidade e, especificamente, a escolha de uma métrica) que aqui sejam consideradas. 

Nesta seção abordaremos algumas das abordagens mais comuns na construção de matrizes de pesos espaciais (W). Iniciaremos a exploração pelas abordagens mais comuns e intuitivas, assentes explicitamente num espaço métrico - ou seja, onde a matriz de pesos deriva diretamente de medidas de distância (euclidiana) no espaço geográfico (cartesiano, da superfície terreste) como forma de estabelecer relações entre objetos. 
Em seguida, serão exploradas abordagens que tiram partido de propriedades topológicas de objetos espaciais representados como polígonos - especificamente, quando estes objetos de análise são uma qualquer divisão territorial do espaço geográfico, como é o caso das delimitações político administrativas. Iremos assim explorar abordagens de contiguidade / adjacência entre polígonos. Por fim, abordaremos alguns exemplos de técnicas mais sofisticadas, que combinam uma ou mais operações espaciais para derivar as relações de vizinhança entre as observações.

In [ ]:
import scipy 

In [ ]:
scipy.__version__

In [ ]:
from pysal.lib import weights


# # https://pysal.org/libpysal/api.html
# from libpysal.weights.contiguity import Queen

# from libpysal.cg import voronoi_frames

## Exemplo (mais frequente): unidades de análise representadas por polígonos 

» Relações topológicas

In [ ]:
shape_BGRI21_Porto_CircEUlar.head(3)

In [ ]:
# Contiguidade Rook

w_rook_shape_BGRI21_Porto_CircEUlar = weights.contiguity.Rook.from_dataframe(shape_BGRI21_Porto_CircEUlar)

In [ ]:
shape_BGRI21_Porto_CircEUlar['HousingPrice'].describe()

In [ ]:
shape_BGRI21_Porto_CircEUlar['HousingPrice'].isna().sum()

### Usar a matriz de pesos espaciais para imputação de dados
NOTAS: <br>
a) como temos unidades espaciais com dados omissos em que (todos) os seus vizinhos também têm dados omissos, uma solução é implementar várias iterações do algoritmo de imputação de dados até obtermos a completa imputação dos dados <br>
b) no caso da solução aqui apresentada, a imputação baseia-se na média dos valores dos vizinhos que têm valores atríbuidos (em cada iteração), mas poderíamos considerar outras abordagens, como a imputação baseada em regressão linear, por exemplo.

In [ ]:


shape_BGRI21_Porto_CircEUlar['HousingPrice_Imputation'] = shape_BGRI21_Porto_CircEUlar['HousingPrice_Ori']


# Apply the function to fill missing values
while(shape_BGRI21_Porto_CircEUlar['HousingPrice_Imputation'].isna().sum() > 0):
    shape_BGRI21_Porto_CircEUlar['HousingPrice_Imputation'] = shape_BGRI21_Porto_CircEUlar.apply(lambda row: utils.utils_ete2425.fill_missing_with_neighborhood_avg(row, w=w_rook_shape_BGRI21_Porto_CircEUlar, shape=shape_BGRI21_Porto_CircEUlar, colname='HousingPrice_Imputation'), axis=1)

In [ ]:
shape_BGRI21_Porto_CircEUlar['HousingPrice'].describe()

In [ ]:
shape_BGRI21_Porto_CircEUlar['HousingPrice_Imputation'].describe()

In [ ]:
shape_BGRI21_Porto_CircEUlar['HousingPrice_Imputation'].isna().sum()

In [ ]:
classi_quant = mc.Quantiles(shape_BGRI21_Porto_CircEUlar['HousingPrice_Imputation'], k=5)
classi_quant

In [ ]:
figura, ax = plt.subplots(figsize = (10,10) )
ax.set_aspect('equal')

shape_MUNICIPIO_Porto.plot(ax=ax, zorder=2, 
                             color='grey', 
                             edgecolor='black', linewidth=2,
                            alpha = 0.3)

shape_BGRI21_Porto_CircEUlar.plot(ax=ax,zorder=1,
                      # marker="s",
                      column='HousingPrice_Imputation', 
                      scheme='Quantiles', 
                      # k=7, 
                      cmap= "cividis", 
                      linewidth=0 )



# Dependência espacial

In [ ]:
# Compute spatial lag

precoM2_lag = weights.lag_spatial(w_rook_shape_BGRI21_Porto_CircEUlar, shape_BGRI21_Porto_CircEUlar['HousingPrice_Imputation'])
shape_BGRI21_Porto_CircEUlar["HousingPrice_Imputation_lag"] = precoM2_lag

In [ ]:
# "Normalizar" (cálculo de z-scores) as variáveis (original e desfasada)
shape_BGRI21_Porto_CircEUlar['HousingPrice_Imputation_std'] = ( shape_BGRI21_Porto_CircEUlar['HousingPrice_Imputation'] - shape_BGRI21_Porto_CircEUlar['HousingPrice_Imputation'].mean() )\
                    / shape_BGRI21_Porto_CircEUlar['HousingPrice_Imputation'].std()
shape_BGRI21_Porto_CircEUlar['HousingPrice_Imputation_lag_std'] = ( shape_BGRI21_Porto_CircEUlar['HousingPrice_Imputation_Imputation_lag'] - shape_BGRI21_Porto_CircEUlar['HousingPrice_Imputation_lag'].mean() )\
                    / shape_BGRI21_Porto_CircEUlar['HousingPrice_Imputation_lag'].std()

In [ ]:
# Moran Plot
f, ax = plt.subplots(1, figsize=(6, 6))
sns.regplot(x='HousingPrice_Imputation_std', y='HousingPrice_Imputation_std_lag_std', 
                ci=None, data=shape_BGRI21_Porto_CircEUlar, line_kws={'color':'r'})
ax.axvline(0, c='k', alpha=0.5)
ax.axhline(0, c='k', alpha=0.5)
ax.set_title('Moran Plot')
plt.show()

## Autocorrelação espacial - Global

In [ ]:
# from pysal.explore import esda
from esda.moran import Moran
from esda.moran import Moran_Local

# https://splot.readthedocs.io/en/latest/api.html
# pysal.viz »» splot [splot.esda] (tem métodos para esda e outras componentes - ver api)
from splot.esda import moran_scatterplot
from splot.esda import lisa_cluster
from splot.esda import plot_moran


In [ ]:
moran_HousingTransactionPrice_WRook = Moran(shape_BGRI21_Porto_CircEUlar['HousingPrice_Imputation'], w_rook_shape_BGRI21_Porto_CircEUlar)
moran_HousingTransactionPrice_WRook.I, moran_HousingTransactionPrice_WRook.p_sim

## Autocorrelação espacial - Local

In [ ]:
# calculate Moran_Local and plot
moranLocal_HousingTransactionPrice_WRook = Moran_Local(shape_BGRI21_Porto_CircEUlar['HousingPrice_Imputation'], w_rook_shape_BGRI21_Porto_CircEUlar)

In [ ]:
figura, ax = moran_scatterplot(moranLocal_HousingTransactionPrice_WRook, p=0.05)
ax.set_xlabel('HousingTransactionPrice')
ax.set_ylabel('Spatial Lag of HousingTransactionPrice')
plt.show()

In [ ]:
lisa_cluster(moranLocal_HousingTransactionPrice_WRook, shape_BGRI21_Porto_CircEUlar, p=0.05, figsize = (9,9))
plt.show()